In [1]:
!pip install -r requirements.txt

  Created wheel for en-core-web-trf: filename=en_core_web_trf-3.2.0-py3-none-any.whl size=460235345 sha256=00b357c3dfa963d38a281a15b888d9ee82a1db2ed7fa4fe1f46a878b12ff5bd2
  Stored in directory: c:\users\alexj\appdata\local\pip\cache\wheels\84\ee\01\cabf292482fe93d0a92849c4567a35c38cde57878a756e3393
Successfully built en-core-web-trf
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0


In [1]:
import pandas as pd
import numpy as np

import nltk
import spacy

#from tqdm.auto import tqdm 
#tqdm.pandas()

In [2]:
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from spacytextblob.spacytextblob import SpacyTextBlob

@Language.factory("language_detector")
def create_language_detector(nlp, name):
    return LanguageDetector(language_detection_function=None)


In [6]:
nlp = spacy.load("en_core_web_sm")

nlp.add_pipe('sentencizer')
nlp.add_pipe('language_detector')
nlp.add_pipe('spacytextblob')

In [3]:
df = pd.read_csv(r"https://www.ecb.europa.eu/press/key/shared/data/all_ECB_speeches.csv?848ea64ce6d77827b5e8e18790878b64",index_col = ["date"] , sep = "|",encoding='utf-8')

In [8]:
df = df.dropna()
df = df[df["contents"]!=" "]

In [7]:
df["nlp"] = df["contents"].apply(lambda x: nlp(x))

In [9]:
df= df[df['nlp'].apply(lambda x: x._.language['language']=="en")]

In [9]:
text = [str(x) for x in list(df["nlp"].sample(1)[0].sents)]

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

sentences = ["I represent the BCE", 
             "the sky is grey", 
             "the sky is blue", 
             "I am happy"]

inputs = tokenizer(text, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]

labels = {0:'neutral', 1:'positive',2:'negative'}
for idx, sent in enumerate(text):
    print(sent, '----', outputs.detach().numpy()[idx])

  The role of the European Union in fostering convergence   Speech by Benoît Cœuré, Member of the Executive Board of the ECB, at the Conference on European Economic Integration (CEEI), Vienna, 26 November 2018 This[1] conference was renamed 14 years ago to reflect the “growing together of Europe”.[2] In my remarks today I would like to focus precisely on these three topics: growth, Europe and togetherness. ---- [ 7.199926  -4.5967913 -5.320073 ]
I will argue that these three elements are needed to accomplish what the Treaty on European Union promises: economic and social cohesion.[3]  I will focus my remarks today on the economies of central, eastern and south-eastern Europe (CESEE), covering both those that are already part of the European Union (EU) and those that are EU candidate countries or potential candidates.[4]  I will start with a brief review of the current state of convergence of CESEE economies, and then explain how three key European policy areas – the completion of the S

In [11]:
score = {0:0, 1:1, 2:-1}
np.vectorize(lambda x: score[x])(np.argmax(outputs.detach().numpy(), axis = 1))

array([ 0,  0,  1,  1,  0, -1,  0,  0, -1, -1,  0,  0, -1,  0, -1, -1,  0,
        0,  0,  0, -1,  0,  0,  0, -1,  0,  0, -1,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,
        0,  1,  0,  0, -1,  0,  0,  0,  1,  1,  0,  0,  0,  1,  1,  1,  1,
        1,  0,  0,  1,  0,  1,  0,  0, -1,  0,  1,  1,  0, -1,  1,  0,  0,
        1,  0,  0,  1,  0, -1,  1,  0,  0,  0,  0,  0, -1,  0,  1,  0,  0,
        1,  1,  0,  1,  0,  1,  1, -1,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0])

In [12]:
[nlp(x)._.polarity for x in text]

[0.15000000000000002,
 0.04265873015873015,
 0.10357142857142858,
 0.7,
 0.25,
 0.06428571428571428,
 0.0,
 0.045833333333333344,
 0.17047619047619045,
 0.06666666666666667,
 0.11000000000000001,
 -0.03333333333333333,
 -0.24999999999999997,
 0.02,
 0.19999999999999998,
 0.05555555555555555,
 0.0,
 0.0861111111111111,
 0.09999999999999999,
 0.0,
 0.008333333333333331,
 0.3,
 0.22142857142857142,
 0.08333333333333334,
 -0.15,
 0.0,
 -0.041666666666666664,
 -0.2,
 0.0,
 0.31,
 0.07142857142857142,
 0.0,
 0.5,
 0.19999999999999998,
 -0.008333333333333331,
 0.0,
 0.2,
 0.08333333333333333,
 0.1361111111111111,
 0.2,
 0.0,
 0.08666666666666666,
 0.43333333333333335,
 0.0,
 0.5,
 0.35,
 -0.4,
 0.13333333333333333,
 0.26666666666666666,
 0.0625,
 0.1,
 0.35,
 0.1,
 -0.014285714285714282,
 0.25,
 0.04583333333333334,
 0.21212121212121213,
 -0.075,
 0.0,
 0.16666666666666666,
 0.19318181818181818,
 0.25,
 0.03333333333333333,
 -0.03571428571428571,
 -0.007440476190476192,
 0.0,
 0.1611688311688

  Monetary policy in Europe - Quo Vadis?   Speech by Willem F. Duisenberg President of the European Monetary Institute London, 27 November 1997 Ladies and Gentlemen, we are in a fortunate position in Europe. We have been able to prepare a new monetary regime for Europe at a time when a consensus about the role of monetary policy exists among central bankers and the large majority of politicians and academics. We are thus able to shape the new institutions accordingly.  Let me first explain what I mean by the current consensus about the role of monetary policy among macroeconomists. Second, having established the state of the art (and it is certainly still true that central banking is more of an art than a science), I shall briefly describe my vision of how monetary policy could ideally be run in Europe, while illustrating my view that the structure of the ESCB and its operating procedures are in line with these standards, and may thus be able to transform that vision into reality.   A 

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
tfidf_vector = v.fit_transform(df["nlp"].apply(lambda x: x.text))


In [23]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


MemoryError: Unable to allocate 12.5 GiB for an array with shape (2351, 1429746) and data type int32

In [21]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("happiness")

'happiness'

In [10]:
def clean_docs(texts, remove_stopwords=True, n_process = 4):
    nlp = spacy.load("en_core_web_sm")
    docs = nlp.pipe(texts, 
                    disable=['parser', 'ner',
                             'lemmatizer', 'textcat'])
    stopwords = nlp.Defaults.stop_words

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [tok for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

In [36]:
aaa = df["contents"].sample(1)
cleaned = clean_docs(aaa, remove_stopwords=True)
cleaned

[' establishment european systemic risk board challenges opportunities  introductory remarks vítor constâncio vice president ecb eurofi financial forum 2010 plenary session implementing de larosière agenda brussels 29 september 2010 european systemic risk board esrb established january 2011 body responsible macro prudential oversight eu financial system mandated actively monitor sources risk financial stability eu member states financial sectors consideration global developments identify risks assess impact financial system prioritise context systemic risk defined legislation esrb risk disruption financial system potential negative consequences internal market real economy esrb issue warnings significant systemic risks issue recommendations policy action address risks  creation esrb response key lessons financial crisis need enhanced macro prudential oversight financial system new policy function value added macro prudential function safeguard financial stability focus detecting assess

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
tfidf_df.sort_values(by=['date','tfidf'], ascending=[True,False]).groupby(['date']).head(10)


,date,term,tfidf
95116433,1997-02-07,hungarian,0.372429
95122846,1997-02-07,nbh,0.319089
95116435,1997-02-07,hungary,0.318382
95102745,1997-02-07,banking,0.178107
95136906,1997-02-07,western,0.166851
...,...,...,...
15093,2021-11-29,fake,0.136210
24442,2021-11-29,misinformation,0.124959
32753,2021-11-29,scientific,0.124293
39863,2021-11-29,world,0.120502


In [43]:
tfidf_vector = v.fit_transform(clean_docs(df["contents"]))

In [49]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=df.index, columns=v.get_feature_names())
tfidf_df = tfidf_df.stack().reset_index()


MemoryError: Unable to allocate 726. MiB for an array with shape (2351, 40467) and data type float64

In [50]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})


MemoryError: Unable to allocate 726. MiB for an array with shape (40467, 2351) and data type float64